# Building an Intelligent Loan application processing using Multi-Agent Systems with Strands Agents, an open source AI agents SDK

## Introduction

In this lab, we are building an Intellign Loan Applicaton Processing using multi-agent systems using the Strands SDK's Agent Graph tool. Throughout this notebook, we'll explore how to create, manage, and leverage networks of AI agents to solve complex problems through collaboration.

By the end of this notebook, you'll understand about:
-  multi-agent hierarchical topologies
- Send messages between agents
- Monitor agent interactions
- Design specialized agent networks for different use cases

Let's begin our exploration of collaborative AI systems!



## Prerequisites

- Python 3.10+
- AWS account
- Anthropic Claude 3.7 enabled on Amazon Bedrock
- IAM role with permissions to use Amazon Bedrock 
- Basic understanding of AI agents and prompt engineering



### Brief Overview of Multi-Agent Systems
Multi-agent systems consist of multiple autonomous agents collaborating to solve complex problems through task distribution, specialization, and parallel processing.

Imagine a multi-agent system like a group of peers collaborating on a project. Each member of the team has an assigned role to help distribute the work of the project, and that work is usually catered to the expertise of that team member. Similarly, in a multi-agent system, each agent is a subject matter expert of some topic and is given relevant tools and resources. When given a project, a coordinator agent can split the work among the group of subject-matter expert agents to distribute and reduce the complexity of each unit of work. Once completed, the work of each agent can be combined through a coordinator agent to complete the project.

Phoenix SDK provides built-in support for creating these systems through the agent_graph tool, allowing developers to move beyond single-agent limitations.




### Key Capabilities
**Explicit Network Topologies:** Define precise communication structures

**Specialized Agent Roles:** Create purpose-built agents with tailored system prompts

**Controlled Information Flow:** Manage how information passes between agents

**Parallel Processing:** Execute agent operations concurrently

**Persistent State:** Maintain long-running agent networks across multiple interactions

**Rich Status Monitoring:** View detailed information about graph structure and message queues



## Core Components and Topology Patterns¶
Agent Graph consists of nodes (agents) connected by edges (communication channels) arranged in specific topologies:

**Nodes (Agents):** Individual AI agents with unique identity and specialized system prompt. Each node:

1. Processes messages independently in its own thread
2. Maintains a private message queue for incoming tasks
3. Has rate limiting to prevent overloading
4. Broadcasts responses to all connected neighbors

**Edges (Connections):** Directed communication channels between agents that define explicit pathways for information flow. They can be:

1. One-way (information flows in one direction)
2. Bidirectional (automatically created in mesh topologies)
3. Explicitly defined to control information routing

                                                                                                 


## Loan Underwriting Process Overview

The loan underwriting process involves systematic evaluation of loan applications through multiple tasks on different domains:

1. **Financial Analysis**
    2. ***Application Intake & Validation***
    3. ***Credit Assessment***
    4. ***Income & Employment Verification***
    5. ***Asset Verification***
    6. ***Property Appraisal*** (if applicable)
7. **Risk Analysis**
    8. ***Risk Assessment***
    9. ***Fraud and misrepresenation***
10. **Compliance & Regulatory Review**
11. **Final Decision & Documentation**


## Topology Patterns:



#### Hierarchical Topology
Tree structure with parent-child relationships, ideal for layered processing and clear reporting lines.

<p align="center">
    <img src="./images/IntellingentLoanTopology.png">
</p>



### Supported Actions
The agent_graph tool supports five primary actions:

1. create: Build a new agent network with specified topology
2. message: Send information to a specific agent in the network
3. status: Check the current state of an agent network
4. list: View all active agent networks
5. stop: Terminate an agent network when it's no longer needed

## Setup and Installation

First, let's make sure we have the Phoenix SDK installed with the agent_graph tool.



In [ ]:
%pip install -q -r requirements.txt --no-cache-dir --force-reinstall

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
!pip freeze | grep boto
!pip freeze | grep agentcore

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
aiobotocore @ file:///home/conda/feedstock_root/build_artifacts/aiobotocore_1741606508148/work
boto3==1.41.3
botocore==1.41.3
opentelemetry-instrumentation-boto==0.59b0
bedrock-agentcore==0.1.7
bedrock-agentcore-starter-toolkit==0.1.24


In [2]:
# Import libraries
import os
import json
import requests
import boto3
import time
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()

### 1 - Create Code for the two agents

In [3]:
![ ! -d "agents" ] && mkdir agents

#### 1.1 - Financial and Credit Analysis expert Agent

Firstly let's write our first agent code to a file locally; this agent will later be deployed to AgentCore runtime.

In [4]:
%%writefile agents/financial_analysis_agent.py
import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
from fastapi import FastAPI
import uvicorn

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

# Global MCP client with lazy initialization
_mcp_client = None


system_prompt = """

                You are the Financial Analysis Manager responsible for comprehensive credit evaluation and income verification.
                Your tasks include:

               Step 1: Credit Analysis
                1. Coordinate credit score analysis with Credit Score Agent
                2. Analyze credit history patterns and trends
                3. Evaluate credit utilization and payment history
                4. Assess credit mix and account age
                5. Identify credit red flags or concerns
                6. Provide consolidated credit assessment summary
                
                Focus Areas:
                - FICO/VantageScore analysis
                - Credit report anomalies
                - Recent credit inquiries
                - Derogatory marks evaluation
                - Credit stability assessment
                
                Provide quantitative scores and qualitative insights for decision-making.

                Step 2: Income Verification
                1. Coordinate income verification through multiple sources
                2. Validate employment status and stability
                3. Verify asset declarations and documentation
                4. Cross-reference financial statements
                5. Identify discrepancies or inconsistencies
                6. Provide comprehensive verification summary
                
                Verification Standards:
                - Income source diversity and stability
                - Employment tenure and position
                - Asset liquidity and ownership
                - Documentation authenticity
                - Financial statement consistency

"""



# Initialize agent with minimal tools first
agent = Agent(
    system_prompt=system_prompt, 
    tools=[],  # Start with no tools, add dynamically
    name="Financial Analysis Agent",
    description="Financial Analysis Agent responsible for comprehensive credit evaluation and income verification. .",
)



a2a_server = A2AServer(
    agent=agent,
    http_url=runtime_url,
    serve_at_root=True
)

@app.get("/ping")
def ping():
    return {"status": "healthy"}

@app.on_event("startup")
async def startup_event():
    """Initialize MCP client on startup"""
    await setup_agent_tools()

app.mount("/", a2a_server.to_fastapi_app())

if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

Overwriting agents/financial_analysis_agent.py


#### 1.2 - Risk Analysis Agent

Now, let's write our second agent code to a local file.

In [5]:
%%writefile agents/ris_analysis_agent.py
import logging
import os
import asyncio
from strands import Agent, tool
from strands.multiagent.a2a import A2AServer
import uvicorn
from fastapi import FastAPI

from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')



system_prompt = """

 You are the Risk Analysis Manager responsible for evaluating risks and detecting potential fraud. Your duties include:

                Step 1: Risk Calculation Agent specialized in quantitative risk modeling:
                1. Calculate probability of default (PD)
                2. Estimate loss given default (LGD)
                3. Assess exposure at default (EAD)
                4. Compute risk-adjusted pricing
                5. Analyze portfolio concentration risks
                6. Generate risk scores and ratings

                Evaluate borrower risk profile. Risk Categories:
                - Credit risk (default probability)
                - Fraud risk (application authenticity)
                - Market risk (economic factors)
                - Operational risk (process failures)
                - Concentration risk (portfolio impact)
                
                Use statistical models for accurate risk quantification.


                Step 2: Fraud Detection  focused on identifying fraudulent applications:
                1. Analyze application data for inconsistencies
                2. Detect synthetic identity fraud
                3. Identify document manipulation or forgery
                4. Flag suspicious behavioral patterns
                5. Cross-reference against fraud databases
                6. Generate fraud risk scores
                
                Use pattern recognition and anomaly detection techniques.


"""

agent = Agent(
    system_prompt=system_prompt, 
    tools=[],
    name="Risk Analysis Agent",
    description="An agent to evaluate risks and detecting potential fraud",
)

host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=agent,
    http_url=runtime_url,
    serve_at_root=True
)

app = FastAPI()

@app.get("/ping")
def ping():
    return {"status": "healthy"}

app.mount("/", a2a_server.to_fastapi_app())

if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

Overwriting agents/ris_analysis_agent.py


Let's write a requirements.txt file with dependencies that are needed for the agent.

In [6]:
%%writefile agents/requirements.txt
boto3==1.40.50
bedrock-agentcore==0.1.7
strands-agents[a2a]
strands-agents-tools
pyyaml
ddgs

Overwriting agents/requirements.txt


### Deploy to AgentCore Runtime

Now, let's deploy this solution into AgentCore Runtime.

#### Setup Cognito User Pool

Before deploy agents, we have to set up a Cognito User Pool, so it can validate users that are accessing our agents, or any other Idenitty provider like Okta, Microsoft Entra ID, etc.

We're going to import a helper class, that has methods to simplify few steps in our workshop. This helper class will import methods responsible to create Cognito User Pool

In [7]:
from helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-east-1_XTm9OXIB7
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_XTm9OXIB7/.well-known/openid-configuration
Client ID: 2i7541qk8ididv8qjsuc1nvif1
Bearer Token: eyJraWQiOiJYTnM2b1V6N05sUk9SSWdObzd4U1REZ2Uyand1K2ZYb3haZmJiVVpBRkcwPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI2NDA4OTRjOC0yMDMxLTcwYmUtYzQ3Mi1mNmI0NzBkNzIxMWYiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9YVG05T1hJQjciLCJjbGllbnRfaWQiOiIyaTc1NDFxazhpZGlkdjhxanN1YzFudmlmMSIsIm9yaWdpbl9qdGkiOiI2MjM4NTUxZC1lOGI2LTQ4MWEtYTY0NS01NDdjNDM3NWI2ODgiLCJldmVudF9pZCI6Ijc5MDc3YzVlLTg0ZmQtNDNkZi1hYTcxLWU1M2U1OGVlNGNhOCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NjQwMjYxMjQsImV4cCI6MTc2NDAyOTcyNCwiaWF0IjoxNzY0MDI2MTI0LCJqdGkiOiJhMjNhMjhmMy00MDcwLTQ4YmItYTlhZi0xOTFkODZlNGI0OTkiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.b9icmeyfxp5wOndmPkVmYE77h9eh2hyaY-FO-cLlr_c77KT7VNy9GJkhv5U2B1uVjLME4PVUaoQd

#### Create IAM Role for the Agents

In [8]:
from helpers.utils import create_agentcore_runtime_execution_role, FINANCIAL_ANALYSIS_ROLE_NAME

execution_role_arn_financial_analysis = create_agentcore_runtime_execution_role(FINANCIAL_ANALYSIS_ROLE_NAME)

✅ Created IAM role: AWSFinancialAnalysisBedrockAgentCoreRole-us-east-1
Role ARN: arn:aws:iam::161615149547:role/AWSFinancialAnalysisBedrockAgentCoreRole-us-east-1
ℹ️ Policy AWSDocsAssistantBedrockAgentCorePolicy-us-east-1 already exists
✅ Attached policy to role
Policy ARN: arn:aws:iam::161615149547:policy/AWSDocsAssistantBedrockAgentCorePolicy-us-east-1


In [9]:
from helpers.utils import create_agentcore_runtime_execution_role, RISK_ANALYSIS_ROLE_NAME

execution_role_arn_risk_analysis = create_agentcore_runtime_execution_role(RISK_ANALYSIS_ROLE_NAME)

✅ Created IAM role: AWSRiskAnalysisBedrockAgentCoreRole-us-east-1
Role ARN: arn:aws:iam::161615149547:role/AWSRiskAnalysisBedrockAgentCoreRole-us-east-1
ℹ️ Policy AWSDocsAssistantBedrockAgentCorePolicy-us-east-1 already exists
✅ Attached policy to role
Policy ARN: arn:aws:iam::161615149547:policy/AWSDocsAssistantBedrockAgentCorePolicy-us-east-1


##### Let's configure and deploy our first agent:

In [10]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime_financial_analysis_agent = Runtime()
financial_analysis_agent_name="aws_financial_assistant"

region = boto_session.region_name

# Configure the deployment
response_financial_analysis_agent = agentcore_runtime_financial_analysis_agent.configure(
    entrypoint="agents/financial_analysis_agent.py",
    execution_role=execution_role_arn_financial_analysis,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=financial_analysis_agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
    protocol="A2A",
)

print("Configuration completed:", response_financial_analysis_agent)

Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/agents/financial_analysis_agent.py, bedrock_agentcore_name=financial_analysis_agent
Memory configured with STM only
Configuring BedrockAgentCore agent: aws_financial_assistant


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only
Found existing memory ID from previous launch: aws_financial_assistant_mem-12Pj9K8K9q


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/.dockerignore
Keeping 'aws_financial_assistant' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/.dockerignore') runtime='None' region='us-east-1' account_id='161615149547' execution_role='arn:aws:iam::161615149547:role/AWSFinancialAnalysisBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True memory_id=None


In [11]:
launch_result_financial_analysis = agentcore_runtime_financial_analysis_agent.launch()
print("Launch completed:", launch_result_financial_analysis.agent_arn)

financial_analysis_agent_arn = launch_result_financial_analysis.agent_arn

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Creating memory resource for agent: aws_financial_assistant
✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: aws_financial_assistant_mem-12Pj9K8K9q...
  Found memory: aws_financial_assistant_mem-12Pj9K8K9q
Found existing memory in cloud: aws_financial_assistant_mem-12Pj9K8K9q
Existing memory has 0 strategies
✅ Using existing STM-only memory
Starting CodeBuild ARM64 deployment for agent 'aws_financial_assistant' to account 161615149547 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: aws_f

✅ Reusing existing ECR repository: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-aws_financial_assistant


Reusing existing CodeBuild execution role: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-e051eb5ff4
Using dockerignore.template with 45 patterns for zip filtering
Uploaded source to S3: aws_financial_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-aws_financial_assistant-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.2s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 1.0s
🔄 INSTALL started (total: 10s)
✅ INSTALL completed in 1.0s
🔄 BUILD started (total: 11s)
✅ BUILD completed in 16.4s
🔄 POST_BUILD started (total: 28s)
✅ POST_BUILD completed in 13.4s
🔄 COMPLETED started (total: 41s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfully in 0m 42s
CodeBuild completed successfully
✅ CodeBuild project configuration saved
Deploying to Bedroc

Launch completed: arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/aws_financial_assistant-Sv0Z6Z7Y1F


In [12]:
status_response = agentcore_runtime_financial_analysis_agent.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: aws_financial_assistant_mem-12Pj9K8K9q...
  Found memory: aws_financial_assistant_mem-12Pj9K8K9q
Retrieved Bedrock AgentCore status for: aws_financial_assistant


Final status: READY


### 3 - Invoking A2A agents

Firstly, let's refresh the auth token:

In [13]:
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

#### 3.1 Getting Agent Cards

In [14]:
import logging
from uuid import uuid4
from urllib.parse import quote

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    logger.info(url)
    # Generate a unique session ID
    session_id = str(uuid4())
    logger.info(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
        'X-Amzn-Trace-Id': f'aws_docs_assistant_{session_id}'
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        logger.info(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching agent card: {e}")
        return None

In [15]:
fetch_agent_card(financial_analysis_agent_arn)

ERROR:__main__:Error fetching agent card: 424 Client Error: Failed Dependency for url: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A161615149547%3Aruntime%2Faws_financial_assistant-Sv0Z6Z7Y1F/invocations/.well-known/agent-card.json


In [ ]:
# Now let's import the dependency packages import boto3
from strands import Agent
from strands_tools import agent_graph
import logging


## Importing dependency packages 

In [ ]:
# Now let's import the dependency packages import boto3
import boto3
import time
import yaml
import os
import logging
import base64
import json
from botocore.config import Config
from typing import List, Dict
import PyPDF2



## Understanding Agent Graph Basics

Agent Graph allows you to create networks of specialized AI agents that can communicate with each other to solve complex problems. Let's understand the core concepts:

- **Graph**: A collection of agents organized in a specific topology
- **Nodes**: Individual agents with specific roles and system prompts
- **Edges**: Communication paths between agents
- **Topologies**: Different network structures (star, mesh, hierarchical)



## Loan Underwriting Workbench

## Building Different Network Topologies

Let's explore different network topologies for various use cases:


###  Hierarchical  - Workbench team

In [ ]:
def read_pdf(file_path):
    """
    Read and extract text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted text from the PDF
    """
    try:
        # Open the PDF file in binary read mode
        with open(file_path, 'rb') as file:
            # Create a PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Get number of pages
            num_pages = len(pdf_reader.pages)
            print(f"Total pages: {num_pages}")
            
            # Extract text from each page
            text = ""
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
                
            return text
    
    except FileNotFoundError:
        return "Error: The file was not found."
    except PyPDF2.errors.PdfReadError:
        return "Error: Invalid PDF file or the file is encrypted."
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
# Example usage
if __name__ == "__main__":
    pdf_path = "data/JoeDoeCreditReport.pdf"  # Replace with your PDF file path
    extracted_JoeDoeCreditReport = read_pdf(pdf_path)
    
    pdf_path = "data/JoeDoeBankStatement.pdf"  # Replace with your PDF file path
    extracted_JoeDoeBankStatement = read_pdf(pdf_path)
    
    pdf_path = "data/JoeDoeBankStatement_2.pdf"  # Replace with your PDF file path
    extracted_JoeDoeBankStatement_2 = read_pdf(pdf_path)
    
    pdf_path = "data/JoeDoePayStub.pdf"  # Replace with your PDF file path
    extracted_JoeDoePayStub = read_pdf(pdf_path)

    pdf_path = "data/JoeDoeIDVerification.pdf"  # Replace with your PDF file path
    extracted_JoeDoeIDVerification = read_pdf(pdf_path)

    pdf_path = "data/JoeDoeTaxes.pdf"  # Replace with your PDF file path
    extracted_JoeDoeTaxes = read_pdf(pdf_path)

    pdf_path = "data/JoeDoeLoanApplication.pdf"  # Replace with your PDF file path
    extracted_JoeDoeLoanApplication = read_pdf(pdf_path)

    pdf_path = "data/JoeDoePropertyInfo.pdf"  # Replace with your PDF file path
    extracted_JoeDoePropertyInfo = read_pdf(pdf_path)
    
    # print("Extracted text:")
    # print(extracted_JoeDoeCreditReport[0:1000])
    # print(extracted_JoeDoeBankStatement[0:1000])
    # print(extracted_JoeDoePayStub[0:1000])

In [ ]:
# Enable debug logs and print them to stderr
# logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

In [ ]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

# Create specialized agents
coordinator = Agent (name= "coordinator",
             model= "us.anthropic.claude-3-5-sonnet-20241022-v2:0", 
             system_prompt= f"""
                You are the Loan Underwriting Supervisor Agent responsible for orchestrating the complete loan underwriting process. Your responsibilities include:

                1. Receive and validate loan applications
                2. Coordinate with manager agents to execute underwriting tasks
                3. Monitor progress and handle escalations
                4. Aggregate results from all assessment domains
                5. Make final loan approval/rejection decisions based on comprehensive analysis
                6. Ensure compliance with lending policies and regulations
                7. Generate final underwriting reports
                
                Process Flow:
                - Start with financial analysis and application validation
                - Then, continue with Risk and Fraud analysis.
                - Delegate tasks to appropriate manager agents
                - Monitor and coordinate parallel processing
                - Collect and analyze results from all domains
                - Apply business rules and lending policies
                - Make final decision and generate documentation
                - Share required information required by each task. 
                
                Decision Criteria:
                - Credit score thresholds
                - Debt-to-income ratios
                - Collateral value
                - Risk assessment scores
                - Regulatory compliance status
             """)



risk_analysis_manager = Agent ( 
             name =  "risk_analysis_manager", 
             model=  "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             system_prompt= f"""
                You are the Risk Analysis Manager responsible for evaluating loan risks and detecting potential fraud. Your duties include:

                1. Coordinate risk scoring and probability analysis
                2. Oversee fraud detection processes
                3. Analyze market and economic risk factors
                4. Evaluate borrower risk profile
                5. Assess collateral and security risks
                6. Provide consolidated risk assessment
                
                Risk Categories:
                - Credit risk (default probability)
                - Fraud risk (application authenticity)
                - Market risk (economic factors)
                - Operational risk (process failures)
                - Concentration risk (portfolio impact)
         
             """
            )
            


            
risk_calculation_Agent = Agent (
             name =  "specialist", 
             model = "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             system_prompt =  f""""
                You are the Risk Calculation Agent specialized in quantitative risk modeling. Your tasks:

                1. Calculate probability of default (PD)
                2. Estimate loss given default (LGD)
                3. Assess exposure at default (EAD)
                4. Compute risk-adjusted pricing
                5. Analyze portfolio concentration risks
                6. Generate risk scores and ratings
                
                Use statistical models and historical data for accurate risk quantification.
             """)
             
           
fraud_detection_agent = Agent (
             name = "specialist", 
             model = "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             system_prompt= f""""
                You are the Fraud Detection Agent focused on identifying fraudulent applications. Your responsibilities:

                1. Analyze application data for inconsistencies
                2. Detect synthetic identity fraud
                3. Identify document manipulation or forgery
                4. Flag suspicious behavioral patterns
                5. Cross-reference against fraud databases
                6. Generate fraud risk scores
                
                Use pattern recognition and anomaly detection techniques.
             """)
        
          
policy_documentation_agent = Agent ( 
             name = "specialist",
             model = "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
             system_prompt= f""""
                    You are the Documentation Agent responsible for loan file management. Your duties:
        
                    1. Compile complete loan documentation
                    2. Ensure document completeness and accuracy
                    3. Generate required disclosures and notices
                    4. Create audit trails and decision logs
                    5. Prepare final loan packages
                    6. Archive documents per retention policies
        
                    Maintain comprehensive documentation for regulatory and audit purposes.
             """)

In [ ]:
# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(coordinator, "coordinator")
builder.add_node(financial_analysis_manager, "financial_analysis_manager")
builder.add_node(risk_analysis_manager, "risk_analysis_manager")
builder.add_node(credit_assessment_agent, "credit_assessment_agent")
builder.add_node(verification_agent, "verification_agent")
builder.add_node(risk_calculation_Agent, "risk_calculation_Agent")
builder.add_node(fraud_detection_agent, "fraud_detection_agent")
builder.add_node(policy_documentation_agent, "policy_documentation_agent")


# Add edges (dependencies)
builder.add_edge("coordinator", "financial_analysis_manager")
builder.add_edge("coordinator", "risk_analysis_manager")
builder.add_edge("coordinator", "policy_documentation_agent")

builder.add_edge("financial_analysis_manager", "credit_assessment_agent")
builder.add_edge("financial_analysis_manager", "verification_agent")

builder.add_edge("risk_analysis_manager", "risk_calculation_Agent")
builder.add_edge("risk_analysis_manager", "fraud_detection_agent")



# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("coordinator")

# Build the graph
graph = builder.build()

In [ ]:
#Execute task on newly built graph
result = graph(f"""
            I will give you an Loan Application submission package for Joe Doe. 
            Process each document, and extract require information to process the Loan application. 
            Keep track of the pending or missing documents
            Provide final Approval recomendation.
            Write the insurance policy and provide the final output

            Retrieve information from the application package:
            {extracted_JoeDoeCreditReport} 
            {extracted_JoeDoeBankStatement}
            {extracted_JoeDoeBankStatement_2}
            {extracted_JoeDoePayStub}
            {extracted_JoeDoeIDVerification}
            {extracted_JoeDoeLoanApplication}
            {extracted_JoeDoePropertyInfo}
            {extracted_JoeDoeTaxes}
            """)
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# Get results from specific nodes
print("\n")
print("=============Expert node results only:======================")
print("============================================================")
print(result.results["expert"].result)

## Conclusion: When to use Agent Graph:
1. For complex workflows with different agent roles
2. When you need persistent agent state
3. For custom communication topologies
4. When you need fine-grained control over message routing

##  Congrats!
you've created a Multi-Agent systems with a hierachical topology to implement an intelligent loan application processing soltuon. 